In [67]:
import pickle
import os
from datetime import datetime
from dateutil import tz

from dateutil import parser

In [68]:
path = '/Users/ggao/Documents/boeing777/boeing/'

In [69]:
# Q1: what's the filter? why they're relevant?
# First tweet text:
# "RT @SheliaBurk: @BillWrh1970 @bfraser747 @KellyannePolls He has good relations with Muslims though, doesn't he? How could people no\u2026 "

In [70]:
file_list = os.listdir(path)
# Just to run a small sample first
file_list = [file_list[0]]

In [71]:
data = []
for i in range(len(file_list)):
    with open(path + file_list[i]) as input_file:
        temp_data = pickle.load(input_file)
    data += temp_data
    

In [77]:
### Need to double check the timezone of 'timestamp_ms' and 'created_at'
### Answer: the time zone is already specified in the 'created_at'

# utc = datetime.utcnow()
for j in range(len(data)):
    ori_time = parser.parse(data[j]['created_at'])

    # Convert the time zone to be EST
    to_zone = tz.gettz('EST')
    data[j]['est_time'] = datetime.strftime(ori_time.astimezone(to_zone), '%Y-%m-%d')
    

In [ ]:
# For each tweet, get the time -- truncated to day? and the tokenized tweet

In [72]:
# Tokenize and clean up the tweets
import re
import nltk
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stopwords_list = stopwords.words('english') + punctuation + ['rt', 'via']

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

# regex_str = r'[a-zA-Z_]{3,}|[A-Z]{2,}'

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', flags = re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', flags = re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    # remove emojis
    emojis_re = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)
    
    tokens = emojis_re.sub(r'', s)
    
    # remove hash-tag and mentions 
    tokens = re.sub(r"(?:\#)" + '|' + r'(?:@[\w_]+)', '', tokens)
    # remove URL
    tokens = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' + '|' + r'<[^>]+>', '', tokens)

    # remove emoticons
    emoticons_re = re.compile(emoticons_str, flags = re.VERBOSE | re.IGNORECASE)
    tokens = emoticons_re.sub(r'', tokens)
    
    
    tokens = tokens_re.findall(tokens)
    
    # Filter non stopwords, and the words starting with @
    tokens = [item for item in tokens if item.lower() not in stopwords_list]
    
    return tokens

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        # If it's emoticon then do not return lowercase
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
# tweet = 'RT @marcobonzanini: this is just an example! :D http://example.com #NLP'


datetime.datetime(2017, 1, 1, 0, 33, 12, tzinfo=tzfile('/usr/share/zoneinfo/EST'))

In [79]:
for j in range(len(data)):
    data[j]['tokenized_tweet'] = preprocess(data[j]['text'], lowercase=True)

In [ ]:
### Next we need to learn and conduct sentiment analysis
# TODO: Learn: https://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/

# Archive

In [32]:
# a through f: [] represents a range
# Is two consecutive characters then use two ranges [a-f][a-f]
re.split(r'[a-f]', 'ajkfhdsakFhweiuhfoi', flags = re.VERBOSE | re.IGNORECASE)

['', 'jk', 'h', 's', 'k', 'hw', 'iuh', 'oi']

In [98]:
# \d{1,5} : digits length 1 to 5
# +: 1 or more
# .* anything
# \w alphanumeric (either letter or number)
re.sub(r'\d{1,5}\s\w+\s\w+\.', ' ', 'adjfjadklsf475 wahsington st.fdasklfdj', )

'adjfjadklsf fdasklfdj'